<h1><font size="6">Bee or Wasp?</font></h1>

![bee_img](https://storage.googleapis.com/kagglesdsdata/datasets%2F842582%2F1437912%2Fkaggle_bee_vs_wasp%2Fbee1%2F10457143913_149e654aaa_n.jpg?GoogleAccessId=databundle-worker-v2@kaggle-161607.iam.gserviceaccount.com&Expires=1599423647&Signature=Xi2gXuRGTFXpu1zjDZWjkjXE8%2FENXgSwWR3g0Lixt%2BF9tU%2FITLalVxY8SFHcogZ95BHm7yJQatXMFAI26Db2UjkJGeQFao3qBJwGO4r0YjM%2BhTrt7b0zHHJkO070%2BoFOe5zYA5ZZJNk5dDEONu%2FrN0H4IZ14XJixr6%2Fad8YaZYR%2BZvPZ9hC83WK8bHSJHOJCIcCdnrKqgKO2aa%2BCLugF29TG%2F21xkk4Q6ORCyM4SHRZR6vH8rH2OCF2GmLofxbZcIoI9IXQR2Jp4sdaet15E6aO4g5GpfdOxHaJv5koOmJ5jqUTNYBA1C7OYM2Oxi2zV1O7jLAAv9lJWdKuoEAQC4A%3D%3D)

![](http://)






This notebook uses dataset published by [George Ray] to show a simple baseline.

Thanks for publishing a great dataset.

[George Ray]:https://www.kaggle.com/jerzydziewierz

## Import libraries

In [ ]:
import os
import gc
import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tqdm import tqdm

print('TensorFlow version: %s' % tf.__version__)

## set configurations and read metadata

In [ ]:
ROOT = '../input/bee-vs-wasp/kaggle_bee_vs_wasp/'
SEED = 42
EPOCHS = 50
BATCH_SIZE = 32
IMG_SIZE = 256


df = pd.read_csv(ROOT+'labels.csv')
df.head()

In [ ]:
def seed_everything(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

## Preprocess and plot

Check the dataset and preprocess it.

In [ ]:
for idx in tqdm(df.index):    
    df.loc[idx,'path']=df.loc[idx,'path'].replace('\\', '/') 
    
df.head()

In [ ]:
labels = list(df['label'].unique())
y = list(df['label'].value_counts())
plt.pie(y, labels=labels, autopct='%1.1f%%', startangle=90)
plt.title('Unique values of the original data')
plt.show()

In [ ]:
labels = list(df['photo_quality'].unique())
x = range(0, 2)
y = list(df['photo_quality'].value_counts())
plt.bar(x, y, tick_label=labels)
plt.title('High quality photos in original data')

plt.show()

In [ ]:
def img_plot(df, label):
    df = df.query('label == @label')
    imgs = []
    for path in df['path'][:9]:
        img = cv2.imread(ROOT+path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        imgs.append(img)
    f, ax = plt.subplots(3, 3, figsize=(15,15))
    for i, img in enumerate(imgs):
        ax[i//3, i%3].imshow(img)
        ax[i//3, i%3].axis('off')
        ax[i//3, i%3].set_title('label: %s' % label)
    plt.show()
    
    
img_plot(df, label='bee')

In [ ]:
img_plot(df, label='wasp')

In [ ]:
img_plot(df, label='insect')

In [ ]:
img_plot(df, label='other')

In [ ]:
# select only high quality photos
df = df.query('photo_quality == 1')
df['label'].value_counts()

In [ ]:
train_df = df.query('is_validation == 0 & is_final_validation == 0').reset_index(drop=True)
val_df = df.query('is_validation == 1').reset_index(drop=True)
test_df = df.query('is_final_validation == 1').reset_index(drop=True)

## Create datasets

In [ ]:
def create_datasets(df, img_size):
    imgs = []
    for path in tqdm(df['path']):
        img = cv2.imread(ROOT+path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size,img_size))
        imgs.append(img)
        
    imgs = np.array(imgs, dtype='float32')
    imgs = imgs / 255.0
    df = pd.get_dummies(df['label'])
    return imgs, df


train_imgs, train_df = create_datasets(train_df, IMG_SIZE)
val_imgs, val_df = create_datasets(val_df, IMG_SIZE)
test_imgs, test_df = create_datasets(test_df, IMG_SIZE)

## Build the model

In [ ]:
def build_model(img_size, n):
    inp = Input(shape=(img_size,img_size,n))
    resnet = tf.keras.applications.ResNet50(input_shape=(img_size,img_size,n), 
                                            weights='imagenet', 
                                            include_top=False)
    x = resnet(inp)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(3, activation='softmax')(x)
    model = tf.keras.Model(inputs=inp, outputs=x) 
    opt = tf.keras.optimizers.SGD(momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model(IMG_SIZE, 3)
model.summary()

In [ ]:
def get_lr_callback(batch_size=32, plot=False):
    start_lr = 3e-4
    def step_decay(epoch):
        drop = 0.5
        epochs_drop = 10.0
        lr = start_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
        return lr
    
    lr_callback = tf.keras.callbacks.LearningRateScheduler(step_decay)
    if plot == True:
        rng = [i for i in range(EPOCHS)]
        y = [step_decay(x) for x in rng]
        plt.plot(rng, y)
        plt.xlabel('epoch', size=14)
        plt.ylabel('learning_rate', size=14)
        plt.title('Training Schedule', size=16)
        plt.show()
        
    return lr_callback


get_lr_callback(plot=True)

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(patience=10, 
                                               verbose=1, 
                                               restore_best_weights=True)

history = model.fit(train_imgs, 
                    train_df, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    callbacks=[es_callback, get_lr_callback(BATCH_SIZE)], 
                    validation_data=(val_imgs, val_df))


pd.DataFrame(history.history)[['accuracy', 'val_accuracy']].plot()
pd.DataFrame(history.history)[['loss', 'val_loss']].plot()
plt.show()

gc.collect()

In [ ]:
model.evaluate(test_imgs, test_df) 